
| 라이브러리            | 역할 설명                                             |
| ---------------- | ------------------------------------------------- |
| 🔧 `pypdfium2`   | PDF 페이지를 \*\*이미지(PIL/Image)\*\*로 렌더링 (OCR 전처리 단계) |
| 🧠 `pytesseract` | 이미지에서 **OCR(텍스트 인식)** 수행 (OCR 본 처리 단계)            |


In [ ]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [ ]:
import pypdfium2
import pytesseract

pdf_reader = pypdfium2.PdfDocument(r"G:\내 드라이브\LLM-RAG-LangChain\제안요청서_바레인.pdf")

for page in pdf_reader:
    image = page.render(scale=2.0).to_pil()
    text = pytesseract.image_to_string(image, lang="eng+kor")
    print(text)

##### **PDF에서 OCR 텍스트를 추출하는 통합 유틸리티 함수**

In [ ]:
from typing import Literal, List
from langchain_core.documents import Document
import pytesseract

def extract_ocr_text_from_pdf(
    pdf_path: str,
    lang: str = "eng+kor",
    method: Literal["pdfium", "pdf2image"] = "pdfium",
    scale: float = 2.0
) -> List[Document]:
    docs = []

    if method == "pdfium":
        import pypdfium2
        pdf = pypdfium2.PdfDocument(pdf_path)
        for i, page in enumerate(pdf):
            image = page.render(scale=scale).to_pil()
            text = pytesseract.image_to_string(image, lang=lang)
            if text.strip():
                docs.append(Document(page_content="[OCR: PDFium]\n" + text.strip(), metadata={"page": i+1, "type": "image_ocr"}))

    elif method == "pdf2image":
        from pdf2image import convert_from_path
        images = convert_from_path(pdf_path, dpi=int(200 * scale))  # scale 반영
        for i, image in enumerate(images):
            text = pytesseract.image_to_string(image, lang=lang)
            if text.strip():
                docs.append(Document(page_content="[OCR: PDF2Image]\n" + text.strip(), metadata={"page": i+1, "type": "image_ocr"}))

    else:
        raise ValueError("지원되지 않는 method입니다. 'pdfium' 또는 'pdf2image' 사용")

    return docs